In [9]:
%%js
$(".repository-content, .container").css("width", "90%");
$("body, #outer-dashboard, .navbar-default").css("background-color", "antiquewhite");
$(".navbar-default").css("background-color", "lightblue");

<IPython.core.display.Javascript object>

In [2]:
%%html
<img src="//localhost:3010/files/img/pydata.png">

In [3]:
%%js
Jupyter.notebook.kernel.comm_manager.register_target(
    'foo_comm_channel',
    function (comm, msg) {
        comm.send({url: window.location.href});
    });

<IPython.core.display.Javascript object>

In [4]:
from urllib.parse import urlparse, parse_qs

import io
import base64
import matplotlib.pyplot as plt
import requests

from ipywidgets import widgets
from IPython.display import display, Javascript
from ipykernel.comm import Comm
from wordcloud import WordCloud, STOPWORDS

In [5]:
debug_widget = widgets.HTML('')
display(debug_widget)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [6]:
def make_wordcloud(term):
    try:
        url = r'https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro=&explaintext=&titles={}'.format(term)
        res = requests.get(url).json()
        text = list(res['query']['pages'].values())[0].get('extract', '').lower()
        wc = WordCloud(stopwords=STOPWORDS, background_color='black', width=600,
                       height=300).generate(text)
        
        sio = io.BytesIO()
        
        plt.figure(figsize=(10, 5))
        plt.imshow(wc)
        plt.axis('off')
        plt.savefig(sio, format='png', bbox_inches='tight')
        html = '<img src="data:image/png;base64, {}"/>'.format(base64.b64encode(sio.getvalue()).decode('utf-8'))
        debug_widget.value = html
    except (IndexError, ValueError):
        debug_widget.value = 'Nothing found'

In [7]:
def parse_url(url):
    q = urlparse(url).query
    return {k.lower(): v[0] for k, v in parse_qs(q).items()}

In [8]:
comm = Comm(target_name='foo_comm_channel')
@comm.on_msg
def handle_msg(msg):
    url = msg['content']['data']['url']
    args = parse_url(url)
    term = args.get('term', '')
    if term:
        make_wordcloud(term)
    else:
        debug_widget.value = '<h2>Nothing to show</h2>'